In [4]:
gas_factor = 30

In [7]:
import pandas as pd
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
import numpy as np
import math

# !!! Determinism !!!
np.random.seed(42)

file_list = [("BLS12", "./bls12/one_off_parallel_1000.csv"),
             ("BN", "./bn/one_off_parallel_1000.csv"),
             ("MNT4", "./mnt4/one_off_parallel_1000.csv"),
            ("MNT6", "./mnt6/one_off_parallel_1000.csv")]

def get_dfs(files):
    results = []
    for file in files:
        (name, path) = file;
        df = pd.read_csv(path)
        keep = df[["modulus_limbs", "group_limbs", "run_microseconds"]].copy()
        results.append(keep)
        
    return results

In [8]:
dataframes = get_dfs(file_list)

In [9]:
dataframes[0].head(15)

,modulus_limbs,group_limbs,run_microseconds
0,4,1,1485
1,4,1,1053
2,4,1,1097
3,10,9,7093
4,4,1,1051
5,4,1,1011
6,13,13,12262
7,4,1,1017
8,4,1,1068
9,15,7,17030


In [10]:
dataframes[1].head(15)

,modulus_limbs,group_limbs,run_microseconds
0,4,1,193
1,4,1,111
2,4,1,115
3,7,5,406
4,4,1,105
5,8,15,565
6,4,1,110
7,4,1,156
8,7,5,357
9,4,1,150


In [10]:
def make_averages(df):
    average = df.groupby(['modulus_limbs', "group_limbs"]).mean()
    average["gas"] = average["run_microseconds"].apply(lambda x: gas_factor * math.ceil(x))
    average = average.copy()
    average.reset_index(inplace=True) 
    average.drop("run_microseconds", axis = 1, inplace = True)
    average = average.groupby(['modulus_limbs']).max()
    average = average.copy()
    average.reset_index(inplace=True) 
    average.drop("group_limbs", axis = 1, inplace = True)

    return average

In [11]:
bls12 = make_averages(dataframes[0])
bls12.head(25)

,modulus_limbs,gas
0,4,30180
1,5,43650
2,6,61080
3,7,81270
4,8,110790
5,9,147090
6,10,182490
7,11,223560
8,12,277620
9,13,332160


In [12]:
bn = make_averages(dataframes[1])
bn.head(25)

,modulus_limbs,gas
0,4,32100
1,5,47430
2,6,66600
3,7,89220
4,8,122400
5,9,164310
6,10,204090
7,11,252780
8,12,315420
9,13,380400


In [13]:
mnt4 = make_averages(dataframes[2])
mnt4.head(25)

,modulus_limbs,gas
0,4,2400
1,5,3540
2,6,4800
3,7,6540
4,8,8670
5,9,11130
6,10,13860
7,11,17130
8,12,21030
9,13,24930


In [14]:
mnt6 = make_averages(dataframes[3])
mnt6.head(25)

,modulus_limbs,gas
0,4,2430
1,5,3570
2,6,4860
3,7,6570
4,8,8910
5,9,11160
6,10,13890
7,11,17160
8,12,21300
9,13,25080


In [15]:
bls12.to_csv("./bls12/one_off_results.csv",index = False)
bn.to_csv("./bn/one_off_results.csv",index = False)
mnt4.to_csv("./mnt4/one_off_results.csv",index = False)
mnt6.to_csv("./mnt6/one_off_results.csv",index = False)